## Conexion a la API de IBM Watson

In [40]:
nlu_apikey = "-AqVH2-6FPhc1fTcVHf9WHQXok1CCHul044GKRgQMi_a"
nlu_url = "https://api.au-syd.natural-language-understanding.watson.cloud.ibm.com/instances/07732151-0ad7-45f7-af60-85562116af50"

In [41]:
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, CategoriesOptions, ConceptsOptions, EmotionOptions, EntitiesOptions, KeywordsOptions, RelationsOptions, SentimentOptions

authenticator = IAMAuthenticator(nlu_apikey)
natural_language_undestanding = NaturalLanguageUnderstandingV1(
    version = '2020-08-01', 
    authenticator = authenticator
)
natural_language_undestanding.set_service_url(nlu_url)

## Importar dataset

In [27]:
import pandas as pd
dataset = pd.read_excel('extraccion_uber_completa_SIN_DUPLICADOS.xlsx')

In [47]:
#dataset solamente con el texto de los tweets
tweets = list(dataset['text'][:100])

## Creación de un dataframe con los tweets y las características de sentimiento y categoría obtenidas a partir de IBM Watson

Análisis de los tweets con el modelo de IBM

In [52]:
output = []
for tweet in tweets:

    response = natural_language_undestanding.analyze(text=tweet, features=Features(keywords=KeywordsOptions(), 
                                                              entities=EntitiesOptions(), 
                                                              categories=CategoriesOptions(),
                                                              emotion=EmotionOptions(),
                                                              sentiment=SentimentOptions())).get_result()
    output.append(response)

Del output del modelo, extraemos las características de sentimiento y categpría (topic?)

In [60]:
score = []
for tweet in output:
    score.append(tweet['sentiment']['document']['score'])

sentiment = []
for tweet in output:
    sentiment.append(tweet['sentiment']['document']['label'])

category = []
for tweet in output:
    category.append(tweet['categories'][0]['label'])

In [64]:
df = pd.DataFrame(list(zip(tweets, score, sentiment, category)), columns=['tweet', 'sentiment_score', 'sentiment', 'topic'])

In [65]:
df

,tweet,sentiment_score,sentiment,topic
0,@Uber_Support hi guys why all my trips this we...,-0.912398,negative,/travel/specialty travel/adventure travel
1,So are we getting Uber eats league 1 or not @S...,0.000000,neutral,/sports/baseball
2,"@footballdaily 2-3 years of Messi, a bit of ca...",-0.807459,negative,/sports/soccer
3,@Prashan32439454 @Uber_Support @rameshjoshi80 ...,0.863584,positive,/science/mathematics/arithmetic
4,"Uber's Gopuff partnership, Drizly deal being p...",-0.788377,negative,/news
...,...,...,...,...
95,@Ligue1_ENG @PSG_English Can I use “Messi” as ...,0.000000,neutral,/real estate/buying and selling homes
96,Doctors Appointment App Like Uber \nhttps://t....,0.000000,neutral,/technology and computing/internet technology/...
97,@SidityDes I trade &amp; Uber on the side. Plu...,0.362398,positive,/real estate/buying and selling homes
98,@Uber_India @Uber @Uber_Support Me and my husb...,-0.791676,negative,/automotive and vehicles/auto parts
